In [1]:
import numpy as np
import itertools

### Datele de intrare, conform query si accesari pe statii

In [190]:
# prima varianta
VA = np.array([[0, 1, 0, 1], [0, 1, 1, 0], [1, 0, 1, 1], [0, 0, 1, 1]]).reshape(4, 4)
acc = np.array([[15, 10, 25], [10, 15, 10], [30, 25, 35], [5, 10, 0]]).reshape(4, 3)

In [173]:
# fix exemplul din curs
VA = np.array([[1, 0, 0, 1], [0, 1, 0, 1], [0, 1, 1, 0], [0, 0, 1, 1]]).reshape(4, 4)
acc = np.array([[10, 20, 30], [0, 15, 0], [3, 25, 0], [15, 5, 10]]).reshape(4, 3)

In [301]:
# ceva care merge
VA = np.array([[1, 1, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1]]).reshape(4, 4)
acc = np.array([[5, 20, 5], [10, 15, 10], [30, 25, 35], [5, 10, 0]]).reshape(4, 3)

In [318]:
# variatiuni pe aceeasi tema, merge si asta
VA = np.array([[1, 1, 0, 0], [0, 1, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1]]).reshape(4, 4)
acc = np.array([[5, 20, 5], [10, 15, 10], [30, 25, 35], [5, 10, 0]]).reshape(4, 3)

### Constructie AA conform afinitatilor

Avem
$$aff(A_i, A_j) := \sum_{k \in K}\sum_{\forall S_l}ref_l(q_k) \cdot acc_l(q_k)$$
unde alegem $ref_l(q_k) = 0$, pentru $\forall l, \forall k$.

In [319]:
def K(i, j):
    iCol = VA[:, i]
    jCol = VA[:, j]
    res = []
    for k in range(0, VA.shape[1]):
        if iCol[k] == 1 and iCol[k] == jCol[k]:
            res += [k]
    return res

In [320]:
def aff(i, j):
    AFF = 0
    for k in range(0, len(K(i, j))):
        for p in range(0, acc.shape[1]):
             AFF = AFF + acc[K(i, j)[k], p]
    return AFF

In [321]:
AA = []
for i in range(0, VA.shape[0]):
    for j in range(0, VA.shape[1]):
        AA += [aff(i, j)]
AA = np.array(AA).reshape(VA.shape[0], VA.shape[1])

In [322]:
AA

array([[120,  30,   0,   0],
       [ 30,  65,   0,  35],
       [  0,   0,   0,   0],
       [  0,  35,   0,  50]])

### Determinarea masurii AG maxime si a permutarii

Cautam 
$$\tau^{*} := \arg \max_{\tau \in S_{\#statii}} AG_\tau = \arg \max_{\tau \in S_{\#statii}} \sum_{i} \sum_{j} aff(A_i, A_j) \cdot \big{[} aff(A_i, A_{j-1}) + aff(A_i, A_{j+1}) \big{]}$$

In [323]:
indexes = list(range(0, AA.shape[0]))

In [324]:
permutations = np.array(list(itertools.permutations(indexes)))

In [325]:
def ag(CA):
    AG = 0
    for i in range(0, CA.shape[0]):
        for j in range(0, CA.shape[1]):
            if j == 1:
                AG = AG + CA[i, j] * CA[i, j + 1]
            elif j == CA.shape[1] - 1:
                AG = AG + CA[i, j] * CA[i, j - 1]
            else:
                AG = AG + CA[i, j] * (CA[i, j - 1] + CA[i, j + 1])
    return AG

In [326]:
AGmax = 0
permutation_index = 0
for i in range(0, permutations.shape[0]):
    CA = AA[0:AA.shape[0], permutations[i]]
    AG = ag(CA)
    if AG > AGmax:
        AGmax = AG
        permutation_index = i

In [327]:
AGmax

19150

In [328]:
permutations[permutation_index]

array([2, 0, 1, 3])

In [329]:
CA = AA[permutations[permutation_index],:]
CA = CA[:, permutations[permutation_index]]
CA

array([[  0,   0,   0,   0],
       [  0, 120,  30,   0],
       [  0,  30,  65,  35],
       [  0,   0,  35,  50]])

### Incercarea partitionarilor, plecand de la CA

$$AQ(q_i) := \{ A_j \ | \ use(q_i, A_j) = 1 \}$$

In [330]:
def AQ(i):
    res = []
    for j in range(0, VA.shape[1]):
        if VA[i, j] == 1:
            res += [j]
    return res

In [331]:
def subseteq(set1, set2):
    for i in range(0, len(set1)):
        if set1[i] not in set2:
            return False
    return True

In general, fie ca avem de calculat $TQ$, ori $BQ$, avem ca
$$(T|B)Q := \{ q_i \ | \ AQ(q_i) \subseteq (T|B)A \}$$

In [332]:
def BTQ(BT):
    res = []
    for i in range(0, VA.shape[0]):
        if subseteq(AQ(i), BT):
            res += [i]
    return res

Costul pe o multime data de query-uri, fie $B$, $T$ ori $O$ se defineste ca fiind:
$$CQ := \sum_{q_i \in (B|T|O)Q} \sum_{\forall S_j} acc_j(q_i)$$

In [333]:
def CQ(BTO):
    cost = 0
    for i in range(0, len(BTO)):
        for j in range(0, acc.shape[1]):
            cost += acc[BTO[i], j]
    return cost

In acest moment, incercam partitionari prin alegerea punctelor pe diagonala principala, maximizand expresia 
$$z := CTQ \cdot CBQ - COQ^2$$

In [334]:
for i in range(0, permutations[permutation_index].shape[0] + 1):
    TA = set(permutations[permutation_index][0:i])
    BA = set(permutations[permutation_index]) - TA
    TA = list(TA); BA = list(BA)
    TQ = BTQ(TA); BQ = BTQ(BA)
    OQ = list(set(permutations[permutation_index]) - set(TQ + BQ))
    CTQ = CQ(TQ); CBQ = CQ(BQ); COQ = CQ(OQ)
    z = CTQ * CBQ - COQ ** 2
    print ("Raport etapa", i)
    print ("Partitionarile sunt", "TA =", TA, "BA =", BA)
    print ("Masura calculata z =", z)
    print ("Multimile de query-uri", "TQ =", TQ, "BQ =", BQ, "OQ =", OQ)
    print ("Costurile", "CTQ =", CTQ, "CBQ =", CBQ, "COQ =", COQ)
    print ("")

Raport etapa 0
Partitionarile sunt TA = [] BA = [0, 1, 2, 3]
Masura calculata z = 0
Multimile de query-uri TQ = [] BQ = [0, 1, 2, 3] OQ = []
Costurile CTQ = 0 CBQ = 170 COQ = 0

Raport etapa 1
Partitionarile sunt TA = [2] BA = [0, 1, 3]
Masura calculata z = 0
Multimile de query-uri TQ = [] BQ = [0, 1, 2, 3] OQ = []
Costurile CTQ = 0 CBQ = 170 COQ = 0

Raport etapa 2
Partitionarile sunt TA = [0, 2] BA = [1, 3]
Masura calculata z = 3600
Multimile de query-uri TQ = [2] BQ = [1, 3] OQ = [0]
Costurile CTQ = 90 CBQ = 50 COQ = 30

Raport etapa 3
Partitionarile sunt TA = [0, 1, 2] BA = [3]
Masura calculata z = 575
Multimile de query-uri TQ = [0, 2] BQ = [3] OQ = [1]
Costurile CTQ = 120 CBQ = 15 COQ = 35

Raport etapa 4
Partitionarile sunt TA = [0, 1, 2, 3] BA = []
Masura calculata z = 0
Multimile de query-uri TQ = [0, 1, 2, 3] BQ = [] OQ = []
Costurile CTQ = 170 CBQ = 0 COQ = 0

